# Missing synchronization data

We have tracking results for quite a lot of sessions for which we have no estimates of ongoing frame rate. Why?

In [1]:
from pathlib import Path
import os, sys

from dotenv import load_dotenv

# Load environmental variables and define paths
load_dotenv()

sys.path.insert(0, str(Path.cwd().parent.parent.parent))
from lib import utils

In [2]:
# Read blocks without ongoing fps estimates from log file
data_path = Path(os.getenv("local_home")) / 'Task_Switching/head_tracking'
log_file = '2023-03-02_DLC_alignsync_15-11-28.log'

with open(data_path / log_file) as f:
    log = f.read().splitlines()

blocks = [x.split(' ')[-2:] for x in log if 'Ongoing fps estimates not found' in x]
print(f"{len(blocks)} blocks with missing data")

7 blocks with missing data


The code below shows that edge detection events exist for only one block (F1607_Block_J4-1)

In [3]:
# Check if edge detection has been performed on any of these blocks
edge_detect_path = data_path / 'edge_detect_test'
no_edge_blocks = []

for current_block in blocks:
    
    target_file = current_block[0] + '_' + current_block[1] + '.csv'
    if (edge_detect_path / target_file).exists():
        print(target_file)
    else:
        no_edge_blocks.append(current_block)        

print(f"No visual stimulus edges for {len(no_edge_blocks)} of {len(blocks)} blocks")

F1607_Block_J4-1.csv
No visual stimulus edges for 6 of 7 blocks


Now lets check if visual stimulus detection has been performed on these blocks. For this, we'll need the full names of the subjects

In [4]:
# Get full names of subjects
ferrets = utils.query_postgres("SELECT id, name FROM task_switch.ferrets;")
ferrets['full_name'] = 'F' + ferrets['id'].astype(str) + '_' + ferrets['name']

# Look for stimulus-triggered frame values (the precursor to edge detection events) for each block remaining
stim_val_path = data_path / 'StimTriggeredFrameVals'
stim_val_missing = []

for current_block in no_edge_blocks:

    idx = ferrets[ferrets['id'] == int(current_block[0][1:])].index     # Add name of subject to path
    full_name = ferrets.loc[idx, 'full_name'].values[0]
    ferret_path = stim_val_path / full_name

    stim_val_files = [i for i in ferret_path.glob('*_' + current_block[1] + '_*')]
    if len(stim_val_files) > 0:
        print(current_block)
    else:
        stim_val_missing.append(current_block)

print(f"No visual stimulus edges for {len(stim_val_missing)} of {len(no_edge_blocks)} blocks")

['F1607', 'Block_J4-15']
['F1607', 'Block_J4-19']
['F1607', 'Block_J4-20']
['F1607', 'Block_J4-21']
['F1607', 'Block_J4-25']
No visual stimulus edges for 1 of 6 blocks


The majority of files are lacking assessment for visual synchronization signals - why? The code below shows that it's not because the video files are missing from the database... 

In [ ]:
query = """ 
    SELECT *
    FROM task_switch.video_files
    WHERE ferret = %(fnum)s
        AND block = %(block)s;
"""

in_videotable = []

for current_block in stim_val_missing:

    block_params = {'fnum':int(current_block[0][1:]), 'block':current_block[1].replace('Block_','')}
    vid_files = utils.query_postgres(query, params=block_params)

    if vid_files.shape[0] > 0:
        in_videotable.append(current_block)
    else:
        print(f"{vid_files['filename'].to_list()}")
        

print(f"{len(in_videotable)} of {len(stim_val_missing)} blocks in video files table")

What about the table containing trial times needed to trigger visual signal capture in the synchronization process. This data is held in the MCS_trials table

In [ ]:
query = """ 
SELECT 
    session_dt, starttime, starttimecorrected
FROM task_switch.mcs_trials_20230219
WHERE session_dt = (
    SELECT datetime
    FROM task_switch.sessions
    WHERE ferret = %(fnum)s 
        AND block = %(block)s
);"""


in_mcs_table = []

for current_block in stim_val_missing:

    block_params = {'fnum':int(current_block[0][1:]), 'block':current_block[1].replace('Block_','')}
    mcs_trials = utils.query_postgres(query, params=block_params)

    if mcs_trials.shape[0] > 0:
        in_mcs_table.append(current_block)
    else:
        print(current_block)

print(f"{len(in_mcs_table)} of {len(stim_val_missing)} blocks in video files table")

Ok, there looks like a lot of potential to extract more data and recover these files - is that possible using the get_vis_stim_in_video? Not generally, as the timestamp data used to trigger video frame data capture uses an old method based on individual csv files containing corrected start times.

If we go all the way back to get the timestamps (using verify_trial)

F1518_Block_J2-6: Auditory trials only
F1518_Block_J2-7: Auditory trials only
F1602_Block_J2-9: Auditory trials only
F1605_Block_
    J2-112: Resolved
    J2-128: Resolved
    J2-134: Resolved
    J2-52: Auditory trials only
    J2-8: Resolved
    J3-9: Resolved
    J3-6: Resolved
    J3-22: Resolved
    J3-10: Resolved
    J3-11: Resolved
    J3-101: Auditory trials only (1 trial)
    J4-10: Trials at locations (9 and 3) without bounding box
    J4-13: Auditory trials only
    J4-15: Auditory trials only
    J4-22: Auditory trials only
    J4-23: Auditory trials only
    J4-24: Auditory trials only
    J4-25: Auditory trials only
    J4-26: Auditory trials only
    J4-30: Auditory trials only
    J4-31: Auditory trials only

F1607
    J2-125: Resolved
    J2-126: Resolved
    J2-128: Auditory trials only (n=2)
    J2-135: Resolved
    J2-17: Auditory trials only
    J2-18: Auditory trials only
    J2-36: Resolved
    J2-38: Auditory trials only
    J2-39: Auditory trials only
    J2-41: Auditory trials only
    J2-43: Auditory trials only
    J2-44: Auditory trials only
    J2-48: Auditory trials only
    J2-49: Auditory trials only
    J2-67: Resolved
    J2-7: Auditory trials only
    J2-77: Auditory trials only
    J2-78: Auditory trials only
    J2-79: Auditory trials only
    J2-80: Auditory trials only
    J2-82: Auditory trials only
    J3-113: Auditory trials only
    J3-114: Auditory trials only
    J3-23: Auditory trials only
    J3-33: Auditory trials only
    J3-54: Auditory trials only
    J3-57: Auditory trials only
    J3-6: Resolved
    J3-80: Auditory trials only
    J3-95: Auditory trials only
    J4-27: Auditory trials only
    J4-33: Auditory trials only
    J5-3: Auditory trials only
    J5-8: Auditory trials only

F1613
    J2-21: Resolved
    J2-46: Resolved
    J2-49: Resolved
    J3-8: Resolved
    J3-99: Auditory trials only

In [ ]:
in_mcs_table

In [ ]:
block_params = {'fnum':1605, 'block':'J2-112'}
mcs_trials = utils.query_postgres(query, params=block_params)
mcs_trials.head()